In [4]:
%run 0.0_Dependance.ipynb

In [5]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

for i in ['Bilirubin','Cholesterol','Albumin', 'Copper','Alk_Phos',
       'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin']:
    if df[i].dtypes != "object":
        Q1 = df[i].quantile(0.25)
        Q3 = df[i].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df["extreme_" + i] = df[i].apply(lambda x: 1 if (x < lower_bound or x > upper_bound) else 0)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

from sklearn.preprocessing import RobustScaler

target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)

scaler = RobustScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
import numpy as np
from joblib import Parallel, delayed
from joblib import dump,load

def load_create_dftop(df_path, n_top):
    df = load(df_path)
    df_sorted = df.sort_values(by="LogLoss_mean", ascending=False)
    df_top = df_sorted.iloc[:n_top, :]
    return df_top

# Fonction pour entraîner le modèle et calculer le log_loss
def compute_log_loss(i, df, X_train, y_train, X_test, y_test):
    dico = df.iloc[i, :-2].to_dict()
    model = XGBClassifier(**dico)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    return log_loss(y_test, probs)

df_score_random_v4_top_1000 = load_create_dftop("df_score_random_v1.joblib", 1000)

# Parallélisation du calcul du log_loss
log_losses = Parallel(n_jobs=-1)(delayed(compute_log_loss)(i, df_score_random_v4_top_1000, X_train, y_train, X_test, y_test) for i in range(1000))

df_score_random_v4_top_1000['Test_log_loss'] = log_losses
df_score_random_v4_top_1000 = df_score_random_v4_top_1000.sort_values(by="Test_log_loss",ascending=True)
dump(df_score_random_v4_top_1000,"df_score_random_v4_top_1000.joblib")

['df_score_random_v4_top_1000.joblib']

In [7]:
df_score_random_v4_top_1000 = load("df_score_random_v4_top_1000.joblib")
df_score_random_v4_top_1000

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std,Test_log_loss
24165,400,6,0.06,0.5,1,0.2,9,0.4,0,multi:softprob,mlogloss,gbtree,-0.428677,0.013106,0.439785
29094,400,6,0.06,0.5,1,0.2,10,0.5,0,multi:softprob,mlogloss,gbtree,-0.428612,0.013848,0.440314
20857,600,6,0.05,0.5,1,0.2,8,0.5,0,multi:softprob,mlogloss,gbtree,-0.427803,0.012694,0.440706
23351,800,6,0.05,0.5,1,0.2,8,0.5,0,multi:softprob,mlogloss,gbtree,-0.427803,0.012694,0.440706
12627,400,5,0.07,0.5,1,0.2,6,0.2,3,multi:softprob,mlogloss,gbtree,-0.430429,0.012988,0.440847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31215,700,6,0.09,0.7,0.75,0.2,6,0.3,2,multi:softprob,mlogloss,gbtree,-0.430228,0.013956,0.454264
30465,500,6,0.09,0.5,0.75,0.2,6,0.6,2,multi:softprob,mlogloss,gbtree,-0.430547,0.015737,0.454295
9184,600,6,0.09,0.7,0.75,0.2,7,0.4,1,multi:softprob,mlogloss,gbtree,-0.429576,0.013691,0.454336
18808,600,6,0.08,0.5,0.75,0.2,7,0.8,0,multi:softprob,mlogloss,gbtree,-0.430698,0.015435,0.454547
